
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [1]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn, GLM

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/williamglazer/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


ArgumentError: ArgumentError: Package GLM not found in current path:
- Run `import Pkg; Pkg.add("GLM")` to install the GLM package.


## Fonctions gobales

In [2]:
"""
    splitdataframe(df::DataFrame, p::Real)

Partitionne en un ensemble d'entraînement et un ensemble de validation un DataFrame.

### Arguments
- `df::DataFrame` : Un DataFrame
- `p::Real` : La proportion (entre 0 et 1) de données dans l'ensemble d'entraînement.

### Détails

La fonction renvoie deux DataFrames, un pour l'ensemble d'entraînement et l'autre pour l'ensemble de validation.

### Exemple

\```
 julia> splitdataframe(df, p.7)
\```

"""
function splitdataframe(df::DataFrame, p::Real)
   @assert 0 <= p <= 1 
    
    n = size(df,1)
    
    ind = shuffle(1:n)
    
    threshold = Int64(round(n*p))
    
    indTrain = sort(ind[1:threshold])
    
    indTest = setdiff(1:n,indTrain)
    
    dfTrain = df[indTrain,:]
    dfTest = df[indTest,:]
    
    return dfTrain, dfTest
    
end

splitdataframe

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [3]:
data = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [4]:
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [5]:
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [6]:
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


#### Exclusion des lignes où :SURVERSE est manquante

In [7]:
surverse_df = dropmissing(data, disallowmissing=true)
first(surverse_df,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [8]:
CSV.write("surverse_df.csv", surverse_df)

"surverse_df.csv"

## Chargement des précipitations

In [9]:
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [10]:
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing


 ### Remplissage des données manquantes
Nous allons tenter de remplir les données manquantes par des moyennes de précipitations lorsque les données sont inconnues pour 2 stations ou plus

In [11]:
# fonction permettant de compter le nombre de valeurs du type "missing" dans une ligne
function countMissing(line)
    count=0
    ind = 0
    for i=1:length(line)
        if (ismissing(line[i]))
            count += 1
            ind = i
        end
    end
    return count, ind
end

countMissing (generic function with 1 method)

In [12]:
# fonction permettant de calculer le moyenne des valeurs d'une ligne peu importe le nombre de "missing"
function meanLine(line)
    values = []
    for i=1:length(line)
        if (!ismissing(line[i]))
            append!(values,line[i])
        end
    end
    return mean(values)
end

meanLine (generic function with 1 method)

In [13]:
# fonction permettant de remplacer toutes les valeurs "missing" d'une ligne avec la valeur donnée en parametre
function replaceMissing(line,value)
    n = length(line)
    for i=1:n
        if (ismissing(line[i]))
            line[i] = value
        end
    end
end

replaceMissing (generic function with 1 method)

### Faire le remplissage des valeurs manquantes

In [14]:
n = length(data[:,:date])
prec = data[:,Not(:date)][:,Not(:heure)]
nbMissing = 0

for row in eachrow(prec)
    nbMissing, ind = countMissing(row)
    if(nbMissing == 1)
        row[ind] = round((convert(Vector{Float64},row[Not(ind)])'*betas[:, :β][ind]))
    end
    # remplacer les lignes qui ont de 2 a 4 missing
    if(nbMissing<5 && nbMissing>1)
        replaceMissing(row,round(meanLine(row)))
    end
end
first(prec,10)

UndefVarError: UndefVarError: betas not defined

In [15]:
prec.heure = data[:,:heure]
prec.date = data[:,:date]
first(prec,5)

,McTavish,Bellevue,Assomption,Trudeau,StHubert,heure,date
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64,Date
1,0,0,0,0,missing,0,2013-05-01
2,0,0,0,0,missing,1,2013-05-01
3,0,0,0,0,missing,2,2013-05-01
4,0,0,0,0,missing,3,2013-05-01
5,0,0,0,0,missing,4,2013-05-01


In [16]:
clean_values = prec[: ,[:date, :heure, :McTavish, :Bellevue, :Assomption, :Trudeau, :StHubert]]
CSV.write("clean_values.csv",clean_values)

"clean_values.csv"

# Calcul des max et des sums par jour

In [17]:
pcp_sum = by(prec, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
first(pcp_sum ,10)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,missing
2,2013-05-02,0,0,0,0,missing
3,2013-05-03,0,0,0,0,missing
4,2013-05-04,0,0,0,0,missing
5,2013-05-05,0,0,0,0,missing
6,2013-05-06,0,0,0,0,missing
7,2013-05-07,0,0,0,0,missing
8,2013-05-08,0,0,0,0,missing
9,2013-05-09,10,0,19,0,missing


#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [18]:
pcp_max = by(prec, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(pcp_max,10)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,missing
2,2013-05-02,0,0,0,0,missing
3,2013-05-03,0,0,0,0,missing
4,2013-05-04,0,0,0,0,missing
5,2013-05-05,0,0,0,0,missing
6,2013-05-06,0,0,0,0,missing
7,2013-05-07,0,0,0,0,missing
8,2013-05-08,0,0,0,0,missing
9,2013-05-09,10,0,19,0,missing


#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que la somme des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [19]:
plot(df, x=:SURVERSE, y=:SUM, Geom.boxplot)

UndefVarError: UndefVarError: df not defined

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que le maximum journalier des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [20]:
plot(df, x=:SURVERSE, y=:MAX, Geom.boxplot)

UndefVarError: UndefVarError: df not defined

## Mise en place des valeurs cherchees 

In [21]:
test = CSV.read("data/test.csv")
df = test[:, [:NO_OUVRAGE, :DATE]]
rename!(df, :DATE=>:date)
for name in names(pcp_max[:, Not(:date)])
    name2 = name
    rename!(pcp_max, name=>Symbol("max" * String(name)))
    rename!(pcp_sum, name2=>Symbol("sum" * String(name2)))
end
df = join(df, pcp_max, on=:date, kind= :inner)
df = join(df, pcp_sum, on=:date, kind= :inner)


,NO_OUVRAGE,date,maxMcTavish,maxBellevue,maxAssomption,maxTrudeau,maxStHubert
,String,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,3260-01D,2019-05-02,12,7,7,8,10
2,3260-01D,2019-05-09,45,26,55,43,32
3,3260-01D,2019-05-10,66,48,50,48,50
4,3260-01D,2019-05-15,2,0,0,0,0
5,3260-01D,2019-05-20,40,5,27,51,38
6,3260-01D,2019-05-23,missing,missing,186,66,99
7,3260-01D,2019-05-24,9,missing,12,6,12
8,3260-01D,2019-05-26,3,missing,21,3,5
9,3260-01D,2019-05-30,7,missing,10,missing,10


In [22]:
dataframe = DataFrame(ouvrage=df.NO_OUVRAGE, x₁=df.sumMcTavish,x₂=df.maxMcTavish,x₃=df.sumBellevue,x₄=df.maxBellevue,x₅=df.sumAssomption,x₆=df.maxAssomption,x₇=df.sumTrudeau,x₈=df.maxTrudeau,x₉=df.sumStHubert,x₁₀=df.maxStHubert)


,ouvrage,x₁,x₂,x₃,x₄,x₅,x₆,x₇,x₈
,String,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,3260-01D,26,12,19,7,15,7,13,8
2,3260-01D,89,45,67,26,77,55,86,43
3,3260-01D,385,66,265,48,286,50,285,48
4,3260-01D,2,2,0,0,0,0,0,0
5,3260-01D,46,40,5,5,49,27,53,51
6,3260-01D,missing,missing,missing,missing,351,186,159,66
7,3260-01D,13,9,missing,missing,22,12,15,6
8,3260-01D,3,3,missing,missing,21,21,5,3
9,3260-01D,7,7,missing,missing,13,10,missing,missing


In [23]:
predictions = []
for row in eachrow(dataframe)
    if row.ouvrage == "4350-01D" 
        predictions = vcat(predictions, GLM.predict(Mvieuxmtl, DataFrame(row[Not(:ouvrage)])))
    end
    if row.ouvrage == "3260-01D"
        predictions = vcat(predictions, GLM.predict(Mrdesp, DataFrame(row[Not(:ouvrage)])))
    end
    if row.ouvrage == "3350-07D"
        predictions = vcat(predictions, GLM.predict(Mahunstic, DataFrame(row[Not(:ouvrage)])))
    end
    if row.ouvrage == "4240-01D"
        predictions = vcat(predictions, GLM.predict(Mpauxt, DataFrame(row[Not(:ouvrage)])))
    end
    if row.ouvrage == "4380-01D"
        predictions = vcat(predictions, GLM.predict(Mverdun, DataFrame(row[Not(:ouvrage)])))
    end
end
predictions = round.(predictions)

UndefVarError: UndefVarError: GLM not defined

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une prediction logistique

In [24]:

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
#n = size(test,1)
#surverse = rand(n) .> .5


# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=predictions)
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.


DimensionMismatch: DimensionMismatch("column length 283 for column(s) ID, and is incompatible with column length 0 for column(s) Surverse")